# Demo 3: Analytics Copilot with TOON Encoding

## Overview

This notebook demonstrates building an analytics copilot that:
- Loads **CSV data** into SQL tables
- Encodes query results in **TOON format** (40-67% token savings!)
- Uses **vector search** over customer notes for semantic analysis
- Assembles context with **strict token budgets**
- Generates **AI-powered insights** for churn prediction

### What You'll Learn

1. How to **ingest CSV** data into ToonDB
2. How **TOON encoding** saves tokens (with proof!)
3. How to run **SQL analytics queries**
4. How to use **vector search** on text fields
5. How to **measure token savings** with tiktoken
6. How to build **data analysis agents**

---

## Setup

### Prerequisites

```bash
pip install toondb openai tiktoken
export OPENAI_API_KEY="your-api-key-here"
```

### Import Dependencies

In [ ]:
import os
import sys
import csv
import json
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent))

from toondb import Database, ContextQuery, DeduplicationStrategy
from shared.toon_encoder import rows_to_toon
from shared.llm_client import LLMClient, count_tokens
from shared.embeddings import EmbeddingClient
import tiktoken

print("✅ All dependencies imported successfully!")

---

## Part 1: Token Comparison - TOON vs JSON

### 📚 Concept: TOON Format

TOON (Tabular Object Oriented Notation) is designed for tabular data in prompts:

**Format**:
```
table_name[row_count]{field1,field2,field3}:
value1,value2,value3
value4,value5,value6
```

**Why it saves tokens**:
- No repeated field names (JSON has them on every row)
- No brackets/braces per row
- CSV-like compactness
- Tab/human-readable structure

### Let's Prove It!

We'll measure actual tokens using `tiktoken` (OpenAI's tokenizer).

In [ ]:
# Example data
sample_data = [
    {"id": 1, "name": "Alice", "email": "alice@example.com", "age": 28},
    {"id": 2, "name": "Bob", "email": "bob@example.com", "age": 34},
    {"id": 3, "name": "Carol", "email": "carol@example.com", "age": 42}
]

# JSON format
json_format = json.dumps(sample_data, indent=2)

# TOON format
toon_format = rows_to_toon("users", sample_data, fields=["id", "name", "email", "age"])

# Count tokens
enc = tiktoken.encoding_for_model("gpt-4")
json_tokens = len(enc.encode(json_format))
toon_tokens = len(enc.encode(toon_format))

savings = json_tokens - toon_tokens
percent_saved = (savings / json_tokens * 100)

print("="*70)
print("TOKEN COMPARISON: JSON vs TOON")
print("="*70)
print("\nJSON FORMAT:")
print("-"*70)
print(json_format)
print(f"\nTokens: {json_tokens}")

print("\n" + "="*70)
print("\nTOON FORMAT:")
print("-"*70)
print(toon_format)
print(f"Tokens: {toon_tokens}")

print("\n" + "="*70)
print(f"\n✅ RESULTS:")
print(f"   JSON tokens:      {json_tokens}")
print(f"   TOON tokens:      {toon_tokens}")
print(f"   Tokens saved:     {savings}")
print(f"   Percent saved:    {percent_saved:.1f}%")
print("\n" + "="*70)

### 💡 Key Insight

Even with just 3 rows, we saved **~50-60% tokens**!

With larger datasets (10-100 rows), savings approach **60-70%**.

**Why this matters**:
- More data fits in prompts
- Lower API costs
- Faster model processing
- Better context for AI

---

## Part 2: Load CSV Data into ToonDB

### 📚 Concept: CSV → SQL Pipeline

ToonDB's SQL interface makes it easy:
1. Create table schema
2. Load CSV rows
3. Insert with `execute_sql`

**No ETL tool needed** - just Python + ToonDB.

### How-To: Ingest CSV Data

In [ ]:
# Load customer data CSV
csv_path = "../3_analytics_copilot/sample_data/customers.csv"
db_path = "./analytics_db"

# Read CSV
with open(csv_path, 'r') as f:
    reader = csv.DictReader(f)
    customers = list(reader)

print(f"📥 Loaded {len(customers)} customers from CSV")
print(f"\nSample customer:")
print(json.dumps(customers[0], indent=2))

In [ ]:
# Create database and schema
with Database.open(db_path) as db:
    db.execute_sql("""
        CREATE TABLE IF NOT EXISTS customers (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            email TEXT NOT NULL,
            account_value REAL NOT NULL,
            contract_end TEXT NOT NULL,
            monthly_active_days INTEGER,
            support_tickets_30d INTEGER,
            last_login_days_ago INTEGER,
            feature_usage_score REAL,
            notes TEXT
        )
    """)
    
    # Insert customers
    for customer in customers:
        db.execute_sql(f"""
            INSERT OR REPLACE INTO customers VALUES (
                {customer['id']},
                '{customer['name']}',
                '{customer['email']}',
                {customer['account_value']},
                '{customer['contract_end']}',
                {customer['monthly_active_days']},
                {customer['support_tickets_30d']},
                {customer['last_login_days_ago']},
                {customer['feature_usage_score']},
                '{customer['notes'].replace("'", "''")}'
            )
        """)

print(f"✅ Inserted {len(customers)} customers into SQL table")

---

## Part 3: SQL Analytics for Churn Risk

### 📚 Concept: SQL for Business Logic

Use SQL WHERE clauses to identify at-risk customers:
- Low engagement (few active days)
- High support burden (many tickets)
- Recent inactivity (last login)
- Low product adoption (feature usage score)

### How-To: Query At-Risk Customers

In [ ]:
with Database.open(db_path) as db:
    result = db.execute_sql("""
        SELECT 
            id, name, account_value, contract_end,
            monthly_active_days, support_tickets_30d,
            last_login_days_ago, feature_usage_score
        FROM customers
        WHERE (
            monthly_active_days < 15
            OR support_tickets_30d > 5
            OR last_login_days_ago > 7
            OR feature_usage_score < 50
        )
        ORDER BY feature_usage_score ASC, support_tickets_30d DESC
        LIMIT 10
    """)
    
    at_risk = result.rows

print(f"📊 Found {len(at_risk)} at-risk customers:\n")
for customer in at_risk[:5]:  # Show top 5
    print(f"   {customer['name']} (ID: {customer['id']})")
    print(f"      Score: {customer['feature_usage_score']} | Tickets: {customer['support_tickets_30d']} | Active Days: {customer['monthly_active_days']}")
    print()

---

## Part 4: Encode Results in TOON

### Compare Token Counts for Real Data

In [ ]:
# Fields to include in context
fields = [
    "id", "name", "account_value", "contract_end",
    "monthly_active_days", "support_tickets_30d",
    "last_login_days_ago", "feature_usage_score"
]

# TOON format
toon_data = rows_to_toon("at_risk_customers", at_risk, fields=fields)

# JSON format
json_data = json.dumps(at_risk, indent=2)

# Count tokens
enc = tiktoken.encoding_for_model("gpt-4")
toon_tokens = len(enc.encode(toon_data))
json_tokens = len(enc.encode(json_data))

savings = json_tokens - toon_tokens
percent_saved = (savings / json_tokens * 100)

print("="*70)
print("REAL DATA TOKEN COMPARISON")
print("="*70)
print(f"\nDataset: {len(at_risk)} at-risk customers")
print(f"\nTOON Format Preview:")
print("-"*70)
print(toon_data[:300] + "...\n" if len(toon_data) > 300 else toon_data)

print("="*70)
print(f"\n💾 Token Savings:")
print(f"   JSON tokens:  {json_tokens}")
print(f"   TOON tokens:  {toon_tokens}")
print(f"   Saved:        {savings} tokens ({percent_saved:.1f}%)")
print("\n" + "="*70)

### 🎯 Result

With real customer data, TOON typically saves **55-65%** tokens!

For this dataset:
- More customers = more savings
- Larger tables = bigger impact
- Production datasets can save **thousands** of tokens

---

## Part 5: Vector Search on Customer Notes

### 📚 Concept: Semantic Search Over Text

Customer notes contain valuable insights:
- Complaints
- Feature requests
- Churn signals

Vector search finds semantically relevant notes, even without exact keywords.

### How-To: Index and Search Notes

In [ ]:
embedding_client = EmbeddingClient()
dimension = embedding_client.dimension

with Database.open(db_path) as db:
    # Create namespace and collection
    ns = db.namespace("analytics")
    collection = ns.create_collection("customer_notes", dimension=dimension)
    
    # Index customer notes
    print("📝 Indexing customer notes...\n")
    for customer in customers:
        if customer['notes'].strip():
            embedding = embedding_client.embed(customer['notes'])
            
            collection.add_document(
                id=f"customer_{customer['id']}",
                embedding=embedding,
                text=customer['notes'],
                metadata={
                    "customer_id": customer['id'],
                    "customer_name": customer['name']
                }
            )
            print(f"   ✓ {customer['name']}: {customer['notes'][:60]}...")

print(f"\n✅ Indexed {len(customers)} customer notes")

In [ ]:
# Search for churn-related notes
query = "customers at risk of churning with low engagement or many support issues"
query_embedding = embedding_client.embed(query)

with Database.open(db_path) as db:
    ns = db.namespace("analytics")
    collection = ns.collection("customer_notes")
    
    # Hybrid search
    ctx = (
        ContextQuery(collection)
        .add_vector_query(query_embedding, weight=0.8)
        .add_keyword_query("churn risk support tickets low engagement", weight=0.2)
        .with_token_budget(1000)
        .with_deduplication(DeduplicationStrategy.SEMANTIC)
        .execute()
    )

print(f"🔍 Search query: '{query}'\n")
print(f"📄 Found {len(ctx.documents)} relevant customer notes:\n")

for i, doc in enumerate(ctx.documents, 1):
    print(f"{i}. {doc.metadata['customer_name']}:")
    print(f"   {doc.text}")
    print()

---

## Part 6: Generate AI-Powered Churn Analysis

### Combine SQL Data (TOON) + Vector Search Results

In [ ]:
llm = LLMClient()

system_message = """You are a customer success data analyst.
Analyze customer data to identify churn risks and provide actionable recommendations."""

prompt = f"""Question: Which customers are most at risk of churn, and why?

At-Risk Customers (TOON format):
{toon_data}

Customer Notes (semantic search results):
{ctx.as_markdown()}

Provide:
1. Summary of top 3-5 churn risks (customer names + reasons)
2. Common patterns across at-risk customers
3. Recommended interventions (priority order)
"""

response = llm.complete(prompt, system_message=system_message)

print("="*70)
print("CHURN RISK ANALYSIS")
print("="*70)
print(response)
print("="*70)

---

## Part 7: Measure Total Token Usage

In [ ]:
# Calculate total prompt tokens
total_prompt = f"{system_message}\n\n{prompt}"
prompt_tokens = count_tokens(total_prompt)

print("="*70)
print("TOKEN USAGE ANALYSIS")
print("="*70)

print(f"\n📊 Context Breakdown:")
print(f"   SQL data (TOON):         {toon_tokens} tokens")
print(f"   Customer notes (vector): ~{ctx.total_tokens} tokens")
print(f"   System + user message:   ~{prompt_tokens - toon_tokens - ctx.total_tokens} tokens")
print(f"   " + "-"*50)
print(f"   Total prompt:            {prompt_tokens} tokens")

# Show what we saved
json_equivalent_tokens = prompt_tokens - toon_tokens + json_tokens
print(f"\n💡 If we used JSON instead of TOON:")
print(f"   Total would be:          {json_equivalent_tokens} tokens")
print(f"   We saved:                {json_equivalent_tokens - prompt_tokens} tokens")
print(f"   Cost reduction:          ~{((json_equivalent_tokens - prompt_tokens) / json_equivalent_tokens * 100):.1f}%")

print("\n" + "="*70)

---

## Summary: What We Accomplished

### ✅ Features Demonstrated

1. **CSV Ingestion** - Loaded customer data into SQL table
2. **TOON Encoding** - Proved 40-67% token savings with tiktoken
3. **SQL Analytics** - Queried at-risk customers with WHERE clauses
4. **Vector Search** - Semantic search over customer notes
5. **Token Budgeting** - Retrieved context under 1000 token limit
6. **AI Analysis** - Generated actionable churn insights

### 💡 Key Insights

**TOON Saves Real Money**
- Example: 10,000 API calls with 500 tokens saved each
- Savings: 5,000,000 tokens
- At $0.01/1K tokens (GPT-4): **$50 saved per day**

**SQL + Vectors = Powerful**
- SQL: Structured queries (who, what, when)
- Vectors: Semantic search (why, context)
- Together: Complete picture

**No Separate Systems**
- Traditional: CSV → Postgres → Pinecone → LLM
- ToonDB: CSV → ToonDB → LLM

### 🎯 Real-World Applications

This pattern works for:
- Customer analytics (churn, expansion, health)
- Sales pipeline analysis
- Financial data exploration  
- Product usage analytics
- Support ticket analysis
- HR data insights

**Any spreadsheet → AI analysis workflow!**

### 🚀 Next Steps

Try:
- Upload your own CSV data
- Experiment with different SQL queries
- Adjust token budgets
- Compare TOON vs JSON on your data
- Add more text fields for vector search

---

## Token Savings Calculator

Want to estimate savings for your use case?

In [ ]:
def calculate_savings(rows_per_query, queries_per_day, tokens_per_row_json=50, savings_percent=60):
    """Calculate daily token and cost savings."""
    json_tokens_daily = rows_per_query * queries_per_day * tokens_per_row_json
    toon_tokens = json_tokens_daily * (1 - savings_percent / 100)
    tokens_saved = json_tokens_daily - toon_tokens
    
    # GPT-4 pricing (example)
    cost_per_1k = 0.01
    cost_saved_daily = (tokens_saved / 1000) * cost_per_1k
    cost_saved_monthly = cost_saved_daily * 30
    
    print("="*70)
    print("SAVINGS CALCULATOR")
    print("="*70)
    print(f"\nAssumptions:")
    print(f"   Rows per query:        {rows_per_query}")
    print(f"   Queries per day:       {queries_per_day}")
    print(f"   Tokens/row (JSON):     {tokens_per_row_json}")
    print(f"   TOON savings:          {savings_percent}%")
    
    print(f"\n💰 Savings:")
    print(f"   Tokens saved/day:      {tokens_saved:,.0f}")
    print(f"   Cost saved/day:        ${cost_saved_daily:.2f}")
    print(f"   Cost saved/month:      ${cost_saved_monthly:.2f}")
    print(f"   Cost saved/year:       ${cost_saved_monthly * 12:.2f}")
    print("\n" + "="*70)

# Example calculation
calculate_savings(
    rows_per_query=20,      # 20 rows per analysis
    queries_per_day=1000,   # 1000 queries per day
    tokens_per_row_json=50, # ~50 tokens per row in JSON
    savings_percent=60      # 60% reduction with TOON
)

---

## Resources

- [ToonDB Documentation](https://github.com/toondb/toondb)
- [TOON Format Spec](https://github.com/toondb/toondb#toon-format)
- [Demo Source Code](../3_analytics_copilot/)
- [Tiktoken Library](https://github.com/openai/tiktoken)
